# Scenario 04: Deterministic Multi-Agent Workflows

**Estimated Time**: 45 minutes

## Learning Objectives
- Design deterministic workflows for agent coordination
- Implement sequential and parallel execution patterns
- Handle conditional branching and error recovery
- Use shared context for data flow between agents

## Prerequisites
- Completed Scenario 01 (Simple Agent + MCP)
- Understanding of async/await patterns in Python

## Part 1: Understanding Deterministic Workflows

### What Are Deterministic Workflows?

Deterministic workflows define a **fixed execution path** for coordinating multiple agents:

- **Predictable**: Same inputs produce same execution order
- **Debuggable**: Clear step-by-step execution trace
- **Testable**: Each step can be tested independently
- **Recoverable**: Built-in error handling and retry logic

### Workflow vs Non-Deterministic Patterns

| Deterministic | Non-Deterministic |
|--------------|------------------|
| Fixed execution order | Agent decides next action |
| Explicit data flow | Implicit message passing |
| Predictable completion | May run indefinitely |
| Easy to test | Harder to test |

### When to Use Deterministic Workflows

- ETL pipelines with agent transformations
- Document processing (extract → analyze → summarize)
- Research workflows (search → analyze → report)
- Approval workflows with human-in-the-loop

## Part 2: Setting Up the Environment

In [3]:
# Load environment and configure paths
import sys
from pathlib import Path

# Add project root to path
project_root = Path("..").resolve()
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# Load environment variables
from dotenv import load_dotenv
load_dotenv(project_root / ".env")

print(f"✅ Project root: {project_root}")

✅ Project root: C:\Users\jonasrotter\OneDrive - Microsoft\Desktop\Jonas Privat\MyCodingProjects\agents-workshop


In [4]:
# Verify imports
import sys
import asyncio
from pathlib import Path

# Ensure we can import from src
project_root = Path.cwd()
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# Import workflow components
from src.workflows import (
    WorkflowEngine,
    WorkflowResult,
    WorkflowContext,
    ErrorStrategy,
    create_workflow,
    WorkflowStep,
    SequentialStep,
    ParallelStep,
    ConditionalStep,
    DataTransform,
)
from src.workflows.steps import (
    AgentStep,
    StepStatus,
    StepResult,
    RetryConfig,
)
from src.workflows.engine import WorkflowBuilder
from src.common.telemetry import setup_telemetry, get_tracer

# Setup telemetry
setup_telemetry()
tracer = get_tracer(__name__)

print("✅ Workflow components imported successfully!")
print(f"\nError strategies: {[s.value for s in ErrorStrategy]}")

✅ Workflow components imported successfully!

Error strategies: ['abort', 'skip', 'retry', 'fallback']


## Part 3: Basic Workflow Structure

Let's start with a simple workflow using custom steps.

In [5]:
# Create a simple custom step
class EchoStep(WorkflowStep):
    """A simple step that echoes input with a prefix."""
    
    def __init__(self, name: str, prefix: str):
        super().__init__(name)
        self.prefix = prefix
    
    async def execute(self, inputs, context):
        import time
        start = time.time()
        
        message = inputs.get("message", "")
        result = f"{self.prefix}: {message}"
        
        return StepResult(
            step_name=self.name,
            status=StepStatus.COMPLETED,
            outputs={"message": result},
            duration_ms=(time.time() - start) * 1000
        )

# Create workflow engine
engine = WorkflowEngine(name="echo_pipeline")

# Add steps
engine.add_step(EchoStep("step1", "[Step 1]"))
engine.add_step(EchoStep("step2", "[Step 2]"))
engine.add_step(EchoStep("step3", "[Step 3]"))

# Execute workflow
result = await engine.execute({"message": "Hello, Workflow!"})

print(f"Status: {result.status.value}")
print(f"Final output: {result.outputs['message']}")
print(f"Duration: {result.duration_ms:.2f}ms")

Status: completed
Final output: [Step 3]: [Step 2]: [Step 1]: Hello, Workflow!
Duration: 0.52ms


In [6]:
# Inspect step results
print("=== Step Results ===")
for name, step_result in result.step_results.items():
    print(f"  {name}: {step_result.status.value}")
    print(f"    Output: {step_result.outputs}")

=== Step Results ===
  step1: completed
    Output: {'message': '[Step 1]: Hello, Workflow!'}
  step2: completed
    Output: {'message': '[Step 2]: [Step 1]: Hello, Workflow!'}
  step3: completed
    Output: {'message': '[Step 3]: [Step 2]: [Step 1]: Hello, Workflow!'}


## Part 4: Sequential Step Composition

Group steps into sequential containers for logical organization.

In [7]:
# Create processing step
class ProcessingStep(WorkflowStep):
    """A step that processes data."""
    
    def __init__(self, name: str, operation: str):
        super().__init__(name)
        self.operation = operation
    
    async def execute(self, inputs, context):
        import time
        start = time.time()
        
        data = inputs.get("data", "")
        
        if self.operation == "uppercase":
            result = data.upper()
        elif self.operation == "reverse":
            result = data[::-1]
        elif self.operation == "count":
            result = f"{data} (length: {len(data)})"
        else:
            result = data
        
        return StepResult(
            step_name=self.name,
            status=StepStatus.COMPLETED,
            outputs={"data": result},
            duration_ms=(time.time() - start) * 1000
        )

# Create a sequential step container
text_pipeline = SequentialStep(
    name="text_processing",
    steps=[
        ProcessingStep("uppercase", "uppercase"),
        ProcessingStep("reverse", "reverse"),
        ProcessingStep("count", "count"),
    ],
    description="Process text through multiple transformations"
)

# Execute
context = WorkflowContext(workflow_name="demo")
result = await text_pipeline.execute({"data": "hello world"}, context)

print(f"Input: 'hello world'")
print(f"Output: '{result.outputs['data']}'")

Input: 'hello world'
Output: 'DLROW OLLEH (length: 11)'


## Part 5: Parallel Step Execution

Execute multiple steps concurrently when they're independent.

In [8]:
import asyncio

# Create steps with simulated delay
class AnalysisStep(WorkflowStep):
    """A step that performs analysis with simulated delay."""
    
    def __init__(self, name: str, analysis_type: str, delay: float = 0.5):
        super().__init__(name)
        self.analysis_type = analysis_type
        self.delay = delay
    
    async def execute(self, inputs, context):
        import time
        start = time.time()
        
        # Simulate analysis work
        await asyncio.sleep(self.delay)
        
        data = inputs.get("text", "")
        
        return StepResult(
            step_name=self.name,
            status=StepStatus.COMPLETED,
            outputs={self.analysis_type: f"Analysis of '{data[:20]}...'"},
            duration_ms=(time.time() - start) * 1000
        )

# Create parallel analysis
parallel_analysis = ParallelStep(
    name="parallel_analysis",
    steps=[
        AnalysisStep("sentiment", "sentiment", 0.3),
        AnalysisStep("entities", "entities", 0.4),
        AnalysisStep("topics", "topics", 0.5),
    ]
)

context = WorkflowContext(workflow_name="parallel_demo")
result = await parallel_analysis.execute(
    {"text": "The quick brown fox jumps over the lazy dog."},
    context
)

print(f"Status: {result.status.value}")
print(f"Duration: {result.duration_ms:.2f}ms (parallel!)")
print("\n=== Outputs ===")
for key, value in result.outputs.items():
    print(f"  {key}: {value}")

Status: completed
Duration: 516.20ms (parallel!)

=== Outputs ===
  sentiment: Analysis of 'The quick brown fox ...'
  entities: Analysis of 'The quick brown fox ...'
  topics: Analysis of 'The quick brown fox ...'


## Part 6: Conditional Branching

Execute different paths based on conditions.

In [9]:
# Define condition function
def is_long_text(inputs, context):
    """Check if text is longer than threshold."""
    text = inputs.get("text", "")
    return len(text) > 100

# Create conditional step
conditional_processing = ConditionalStep(
    name="length_check",
    condition=is_long_text,
    then_step=ProcessingStep("summarize", "count"),  # Long text
    else_step=ProcessingStep("expand", "uppercase"),  # Short text
    description="Process differently based on text length"
)

# Test with short text
context = WorkflowContext(workflow_name="conditional_demo")
short_result = await conditional_processing.execute(
    {"text": "short", "data": "short"},
    context
)
print(f"Short text result: {short_result.outputs}")

# Test with long text
long_text = "A" * 150
long_result = await conditional_processing.execute(
    {"text": long_text, "data": long_text},
    context
)
print(f"Long text result: {long_result.outputs.get('data', '')[:50]}...")

Short text result: {'data': 'SHORT'}
Long text result: AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...


## Part 7: Data Transformations

Transform data between steps.

In [10]:
# Create data transform
def extract_keywords(inputs):
    """Extract simple keywords from text."""
    text = inputs.get("text", "")
    # Simple keyword extraction (in real app, use NLP)
    words = text.lower().split()
    keywords = [w for w in words if len(w) > 4]
    return {"keywords": keywords[:5]}

transform_step = DataTransform(
    name="extract_keywords",
    transform=extract_keywords,
    description="Extract keywords from text"
)

context = WorkflowContext(workflow_name="transform_demo")
result = await transform_step.execute(
    {"text": "The artificial intelligence revolution is transforming industries worldwide."},
    context
)

print(f"Keywords: {result.outputs['keywords']}")

Keywords: ['artificial', 'intelligence', 'revolution', 'transforming', 'industries']


## Part 8: Error Handling Strategies

Configure how the workflow responds to failures.

In [11]:
# Create a failing step
class FailingStep(WorkflowStep):
    """A step that fails on certain inputs."""
    
    def __init__(self, name: str, fail_on: str):
        super().__init__(name)
        self.fail_on = fail_on
    
    async def execute(self, inputs, context):
        data = inputs.get("data", "")
        
        if self.fail_on in data:
            return StepResult(
                step_name=self.name,
                status=StepStatus.FAILED,
                error=f"Found forbidden value: {self.fail_on}"
            )
        
        return StepResult(
            step_name=self.name,
            status=StepStatus.COMPLETED,
            outputs={"data": f"processed: {data}"}
        )

In [12]:
# Test ABORT strategy (default)
from src.workflows.engine import ErrorConfig

abort_engine = WorkflowEngine(
    name="abort_test",
    error_config=ErrorConfig(strategy=ErrorStrategy.ABORT)
)
abort_engine.add_step(EchoStep("step1", "[1]"))
abort_engine.add_step(FailingStep("step2", "error"))
abort_engine.add_step(EchoStep("step3", "[3]"))

result = await abort_engine.execute({"message": "test", "data": "error here"})
print(f"ABORT Strategy:")
print(f"  Status: {result.status.value}")
print(f"  Error: {result.error}")
print(f"  Steps completed: {len([r for r in result.step_results.values() if r.succeeded])}")

ABORT Strategy:
  Status: failed
  Error: Step 'step2' failed: Failed after 1 attempts: Found forbidden value: error
  Steps completed: 1


In [13]:
# Test SKIP strategy
skip_engine = WorkflowEngine(
    name="skip_test",
    error_config=ErrorConfig(strategy=ErrorStrategy.SKIP)
)
skip_engine.add_step(EchoStep("step1", "[1]"))
skip_engine.add_step(FailingStep("step2", "error"))
skip_engine.add_step(EchoStep("step3", "[3]"))

result = await skip_engine.execute({"message": "test", "data": "error here"})
print(f"SKIP Strategy:")
print(f"  Status: {result.status.value}")
print(f"  Steps completed: {len([r for r in result.step_results.values() if r.succeeded])}")
print(f"  Final message: {result.outputs.get('message', 'N/A')}")

SKIP Strategy:
  Status: completed
  Steps completed: 2
  Final message: [3]: [1]: test


In [14]:
# Test FALLBACK strategy
fallback_engine = WorkflowEngine(
    name="fallback_test",
    error_config=ErrorConfig(
        strategy=ErrorStrategy.FALLBACK,
        fallback_value="default_value"
    )
)
fallback_engine.add_step(EchoStep("step1", "[1]"))
fallback_engine.add_step(FailingStep("step2", "error"))
fallback_engine.add_step(EchoStep("step3", "[3]"))

result = await fallback_engine.execute({"message": "test", "data": "error here"})
print(f"FALLBACK Strategy:")
print(f"  Status: {result.status.value}")
print(f"  Fallback used: {'fallback' in result.outputs}")

FALLBACK Strategy:
  Status: completed
  Fallback used: True


## Part 9: Retry Configuration

Configure automatic retries for flaky steps.

In [15]:
# Create a flaky step that sometimes fails
import random

class FlakyStep(WorkflowStep):
    """A step that randomly fails."""
    
    attempt_count = 0
    
    def __init__(self, name: str, success_rate: float = 0.3):
        retry = RetryConfig(max_attempts=5, delay_seconds=0.1)
        super().__init__(name, retry=retry)
        self.success_rate = success_rate
    
    async def execute(self, inputs, context):
        FlakyStep.attempt_count += 1
        
        if random.random() > self.success_rate:
            return StepResult(
                step_name=self.name,
                status=StepStatus.FAILED,
                error=f"Random failure (attempt {FlakyStep.attempt_count})"
            )
        
        return StepResult(
            step_name=self.name,
            status=StepStatus.COMPLETED,
            outputs={"result": f"Success after {FlakyStep.attempt_count} attempts!"}
        )

# Reset counter
FlakyStep.attempt_count = 0

# Create workflow with retry
retry_engine = WorkflowEngine(
    name="retry_test",
    error_config=ErrorConfig(strategy=ErrorStrategy.RETRY)
)
retry_engine.add_step(FlakyStep("flaky_step", success_rate=0.3))

result = await retry_engine.execute({})
print(f"Retry Result:")
print(f"  Status: {result.status.value}")
print(f"  Total attempts: {FlakyStep.attempt_count}")
if result.succeeded:
    print(f"  Result: {result.outputs.get('result')}")
else:
    print(f"  Error: {result.error}")

Retry Result:
  Status: completed
  Total attempts: 1
  Result: Success after 1 attempts!


## Part 10: Using the Workflow Builder

Fluent API for building workflows.

In [16]:
# Create mock agents for demonstration
class MockAgent:
    def __init__(self, name: str, response: str):
        self.name = name
        self.response = response
    
    async def run(self, prompt: str) -> str:
        return f"{self.response} | Input: {prompt[:50]}..."

# Create workflow using builder
workflow = (
    WorkflowBuilder("research_pipeline")
    .with_description("Research and summarize a topic")
    .with_agent("researcher", MockAgent("researcher", "Research findings"))
    .with_agent("analyzer", MockAgent("analyzer", "Analysis results"))
    .with_agent("summarizer", MockAgent("summarizer", "Executive summary"))
    .add_agent_step(
        name="research",
        agent="researcher",
        prompt="Research the following topic: {topic}",
        outputs=["findings"]
    )
    .add_agent_step(
        name="analyze",
        agent="analyzer",
        prompt="Analyze these findings: {findings}",
        outputs=["analysis"]
    )
    .add_agent_step(
        name="summarize",
        agent="summarizer",
        prompt="Summarize this analysis: {analysis}",
        outputs=["summary"]
    )
    .on_error(ErrorStrategy.ABORT)
    .build()
)

# Execute
result = await workflow.execute({"topic": "AI agents in production"})

print(f"Status: {result.status.value}")
print(f"Duration: {result.duration_ms:.2f}ms")
print(f"\n=== Output ===")
print(f"Summary: {result.outputs.get('summary', 'N/A')}")

Status: completed
Duration: 0.96ms

=== Output ===
Summary: Executive summary | Input: Summarize this analysis: Analysis results | Input:...


## Part 11: Workflow Validation

Validate workflows before execution.

In [17]:
# Create an invalid workflow
invalid_workflow = WorkflowEngine(name="invalid")

# Empty workflow
errors = invalid_workflow.validate()
print("Empty workflow errors:", errors)

# Add step referencing unknown agent
invalid_workflow.add_step(AgentStep(
    name="step1",
    agent_name="unknown_agent",
    prompt_template="Hello",
    output_vars=["result"]
))

errors = invalid_workflow.validate()
print("Missing agent errors:", errors)

Empty workflow errors: ['Workflow has no steps']
Missing agent errors: ["Step 'step1' references unregistered agent 'unknown_agent'"]


## Part 12: Hands-On Exercise

### Exercise: Build a Document Processing Workflow

Create a workflow that:
1. Extracts text from a document
2. Analyzes sentiment (parallel: positive, negative, neutral)
3. Generates a summary
4. Adds metadata

In [18]:
# Exercise: Complete this workflow

# Step 1: Create extraction step
class ExtractTextStep(WorkflowStep):
    async def execute(self, inputs, context):
        document = inputs.get("document", "")
        # Simulate text extraction
        text = f"Extracted text from: {document}"
        return StepResult(
            step_name=self.name,
            status=StepStatus.COMPLETED,
            outputs={"text": text}
        )

# Step 2: Create sentiment analysis steps (parallel)
# TODO: Create SentimentStep class
class SentimentStep(WorkflowStep):
    def __init__(self, name: str, sentiment_type: str):
        super().__init__(name)
        self.sentiment_type = sentiment_type
    
    async def execute(self, inputs, context):
        text = inputs.get("text", "")
        # TODO: Implement sentiment scoring
        score = 0.5  # Placeholder
        return StepResult(
            step_name=self.name,
            status=StepStatus.COMPLETED,
            outputs={f"{self.sentiment_type}_score": score}
        )

# Step 3: Build the workflow
doc_workflow = WorkflowEngine(name="document_processing")

# TODO: Add steps
doc_workflow.add_step(ExtractTextStep("extract"))
doc_workflow.add_step(ParallelStep(
    name="sentiment_analysis",
    steps=[
        SentimentStep("positive", "positive"),
        SentimentStep("negative", "negative"),
        SentimentStep("neutral", "neutral"),
    ]
))

# Execute
result = await doc_workflow.execute({"document": "quarterly_report.pdf"})
print(f"Status: {result.status.value}")
print(f"Outputs: {result.outputs}")

Status: completed
Outputs: {'text': 'Extracted text from: quarterly_report.pdf', 'positive_score': 0.5, 'negative_score': 0.5, 'neutral_score': 0.5}


## Summary

In this scenario, you learned:

1. **Deterministic Workflows**: Fixed execution paths for multi-agent coordination
2. **Step Types**: Sequential, parallel, conditional, and transform steps
3. **Error Handling**: Abort, skip, retry, and fallback strategies
4. **Data Flow**: Passing outputs between steps
5. **Workflow Builder**: Fluent API for workflow construction
6. **Validation**: Checking workflow configuration before execution

## Key Takeaways

- Deterministic workflows provide predictability and debuggability
- Use sequential steps for dependent operations
- Use parallel steps for independent operations
- Configure error strategies based on your reliability requirements
- Always validate workflows before execution

## Next Steps

- **Scenario 5**: Declarative agent configuration with YAML
- **Scenario 6**: Moderated agent discussions

## Resources

- [Data Model](specs/001-agentic-patterns-workshop/data-model.md)
- [Workflow Patterns](https://www.enterpriseintegrationpatterns.com/)